In [1]:
# Running %env without any arguments
# lists all environment variables

# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
%env CUDA_VISIBLE_DEVICES = 0

import numpy as np
import pandas as pd
import io
import time
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave
import tensorflow as tf
from tensorflow.python.platform import tf_logging
from tensorflow.contrib import layers
from tensorflow.contrib.training import add_gradients_summaries
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.training import optimizer as tf_optimizer
from tensorflow.python.ops import variables as tf_variables
import os.path
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets import inception
import inception_preprocessing
import logging

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

env: CUDA_VISIBLE_DEVICES=0


In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'
PRETRAINED_MODEL_PATH = DATASET_PATH + 'inception-v3/20160828/inception_v3.ckpt'
#PRETRAINED_MODEL_PATH = DATASET_PATH + 'logs/before/inception_v3_model.ckpt-810491'
LOG_PATH = DATASET_PATH + 'logs/'
TRAIN_PATH = DATASET_PATH + 'Split1/Train/'
VAL_PATH = DATASET_PATH + 'Split1/Validation/'
TEST_PATH = DATASET_PATH + 'Test/'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'
BATCH_SIZE = 128#256

# total_batch_size is BATCH_SIZE * ACCUMULATE_STEP
ACCUMULATE_STEP = 4

IMAGE_WIDTH = 180
IMAGE_HEIGHT = 180
NUM_CLASS = 5270
# validation examples num: 2319624
# train examples num: 10051704
# total step: 157057
TOTAL_EXAMPLES = 10051704
# validation num = 2319624
VAL_CHECK_FREQ = 50
NUM_EPOCHES = 7
INPUT_THREADS = 6
EPOCHES_OVER = 7
#Learning rate information and configuration (Up to you to experiment)
# initial_learning_rate = 0.000003#0.00001
# learning_rate_decay_factor = 0.94
initial_learning_rate = 0.001#0.00001
learning_rate_decay_factor = 0.6
num_epochs_before_decay = 1
#Know the number steps to take before decaying the learning rate and batches per epoch
moving_average_decay = 0.9
momentum = 0.8
num_steps_per_epoch = TOTAL_EXAMPLES / (BATCH_SIZE * ACCUMULATE_STEP) + 1
decay_steps = int(num_epochs_before_decay * num_steps_per_epoch / 6)

In [3]:
# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler(DATASET_PATH + 'tensorflow_inception_160_train.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)

In [4]:
class MiniDataSet(object):
    def __init__(self, file_path_pattern, category_level_csv, num_examples, num_classes, is_training = True, min_after_dequeue=2000, batch_size = BATCH_SIZE, num_epochs = NUM_EPOCHES, num_reader = INPUT_THREADS):
        super(MiniDataSet, self).__init__()
        self._num_examples = num_examples
        self._num_classes = num_classes
        self._file_path_pattern = file_path_pattern
        self._category_level_csv = category_level_csv
        self._num_reader = num_reader
        self._batch_size = batch_size
        self._num_epochs = num_epochs
        self._min_after_dequeue = min_after_dequeue
        self._is_training = is_training
        
    def get_category_description_from_csv(self, level = 0):
        category_map = dict()
        csv = pd.read_csv(self._category_level_csv).values
        for row in csv:  
            category_id, levels = row[0], row[1:]
            category_map[category_id] = levels[level]
        return category_map

    def create_dataset(self):
        opts = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.ZLIB)
        reader = lambda : tf.TFRecordReader(options=opts)
        keys_to_features = {
            'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
            'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
            # notice that we don't have this feature in our TFRecord, so always default provided
            'format': tf.FixedLenFeature([], tf.string, default_value='jpg'),
            'category_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
        }

        items_to_handlers = {
            # automated decode image from features in FixedLenFeature
            'image': slim.tfexample_decoder.Image(image_key='img_raw', format_key='format'),
            'label': slim.tfexample_decoder.Tensor('category_id'),
        }

        decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

        labels_to_name_dict = self.get_category_description_from_csv()

        self._dataset = slim.dataset.Dataset(
            data_sources = self._file_path_pattern,
            decoder = decoder,
            reader = reader,
            # num_readers = 8,
            num_samples = self._num_examples,
            #num_classes = self._num_classes,
            #labels_to_name = labels_to_name_dict,
            items_to_descriptions = None)
        
        # notice that DatasetDataProvider can automate shuffle the examples by ParallelReader using its RandomShuffleQueue
        self._data_provider = slim.dataset_data_provider.DatasetDataProvider(
            self._dataset,
            num_readers = self._num_reader,
            shuffle = True, # default is True
            num_epochs = self._num_epochs,
            common_queue_capacity = self._min_after_dequeue + 3 * self._batch_size,
            common_queue_min = self._min_after_dequeue,
            scope = self._is_training and 'train_files' or 'validation_files')
        
        return self._data_provider.get(['image', 'label'])
        

In [5]:
def preprocess_for_inception(input_image, is_training = True):
    # inception_v3.default_image_size = 299
    return inception_preprocessing.preprocess_image(input_image, 160, 160, True)#is_training)

In [6]:
def cvt_csv2tfrecord():
    count = 0
    category_map = dict()
    csv = pd.read_csv(CATEGORY_NAME_PATH).values
    for row in csv:  
        category_id, _ = row[0], row[1:]
        category_map[category_id] = count
        count += 1
    return category_map

In [7]:
def one_hot_process(org_label, map_table, num_classes):
    return tf.one_hot(map_table.lookup(tf.as_string(org_label)), num_classes, axis=-1)

In [8]:
def my_create_train_op(total_loss, optimizer, summarize_gradients = False, accumulate_factor=None):
    global_step = tf.train.get_or_create_global_step()

    update_ops = set(ops.get_collection(ops.GraphKeys.UPDATE_OPS))

    # Make sure update_ops are computed before total_loss.
    if update_ops:
        with ops.control_dependencies(update_ops):
            barrier = control_flow_ops.no_op(name='update_barrier')
    total_loss = control_flow_ops.with_dependencies([barrier], total_loss)

    variables_to_train = tf_variables.trainable_variables()

    # initialized with 0s
    accum_vars = [tf.Variable(tf.zeros_like(tv.initialized_value()), trainable=False) for tv in variables_to_train]
    zero_ops = [tv.assign(tf.zeros_like(tv)) for tv in accum_vars]

    # Calls the compute_gradients function of the optimizer to obtain... the list of gradients
    grads = optimizer.compute_gradients(
      total_loss,
      variables_to_train,
      gate_gradients=tf_optimizer.Optimizer.GATE_OP,
      aggregation_method=None,
      colocate_gradients_with_ops=False)

    ## Adds to each element from the list you initialized earlier with zeros its gradient (works because accum_vars and grads are in the same order)
    if accumulate_factor is not None:   
        total_loss = array_ops.check_numerics(total_loss, 'LossTensor is inf or nan')
        with tf.control_dependencies([total_loss]):
            accum_ops = [accum_vars[i].assign_add(gv[0]) for i, gv in enumerate(grads) if gv[0] is not None]

        ## Define the training step (part with variable value update)
        accumulate_grads = [(tf.multiply(accum_vars[i], accumulate_factor), gv[1]) for i, gv in enumerate(grads) if gv[0] is not None]
        #accumulate_grads = [(accum_vars[i], gv[1]) for i, gv in enumerate(grads) if gv[0] is not None]
    else:
        accum_ops = tf.ops.no_op
    
    if accumulate_factor is not None: 
        # Summarize gradients.
        if summarize_gradients:
            with ops.name_scope('summarize_grads'):
                add_gradients_summaries(accumulate_grads)
        grad_updates = optimizer.apply_gradients(accumulate_grads, global_step=global_step)
    else:
        # Summarize gradients.
        if summarize_gradients:
            with ops.name_scope('summarize_grads'):
                add_gradients_summaries(grads)
        grad_updates = optimizer.apply_gradients(grads, global_step=global_step)

    with ops.name_scope('train_op'):
        # Ensure the train_tensor computes grad_updates.
        train_op = control_flow_ops.with_dependencies([grad_updates], total_loss)

    # Add the operation used for training to the 'train_op' collection
    train_ops = ops.get_collection_ref(ops.GraphKeys.TRAIN_OP)
    if train_op not in train_ops:
        train_ops.append(train_op)

    return train_op, accum_ops, zero_ops

In [9]:
def_graph = tf.Graph()
with def_graph.as_default() as graph:
    def train_step(input_examples, one_hot_labels):   
        with slim.arg_scope(inception.inception_v3_arg_scope()):
            # here logits is the pre-softmax activations
            logits, end_points = inception.inception_v3(
                input_examples,
                num_classes = NUM_CLASS,
                is_training = True)

            # Create the global step for monitoring the learning_rate and training.
            # since supervisor will also create one global_step, so we create n advance in order to feed into exponential_decay
            global_step = tf.train.get_or_create_global_step(graph = graph)
            
            #variables_to_restore = slim.get_variables_to_restore()
            variables_to_restore = slim.get_variables_to_restore(exclude = ['InceptionV3/Logits', 'InceptionV3/AuxLogits'])
            #variables_to_restore_from_checkpoint = slim.get_variables_to_restore(exclude = variables_to_exclude)
            # Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced, e.x. label smothing
            loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = logits)
            aux_loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = end_points['AuxLogits'], weights=0.2)
            total_loss = tf.losses.get_total_loss()    # obtain the regularization losses as well

           

            #Define your exponentially decaying learning rate
            lr = tf.train.exponential_decay(
                learning_rate = initial_learning_rate,
                global_step = global_step,
                decay_steps = decay_steps,
                decay_rate = learning_rate_decay_factor,
                staircase = True)

            #Now we can define the optimizer that takes on the learning rate
            optimizer = tf.train.AdamOptimizer(learning_rate = lr)
            #optimizer = tf.train.MomentumOptimizer(learning_rate = lr, momentum=momentum)
            
            moving_average_variables = slim.get_model_variables()
            variable_averages = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
            # Use an alternative set of update ops in addition to the default updates:
            tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, variable_averages.apply(moving_average_variables))

            #Create the train_op.
            accumulate_factor = tf.constant([1./ACCUMULATE_STEP], name='accumulate_factor')
            train_op, accum_ops, zero_ops = my_create_train_op(total_loss, optimizer, False, accumulate_factor)
            #Create the train_op.
            #train_op = slim.learning.create_train_op(total_loss, optimizer, summarize_gradients=False)

            #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
            predictions = tf.argmax(end_points['Predictions'], 1)
            probabilities = end_points['Predictions']
            accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, tf.argmax(one_hot_labels, 1))
            metrics_op = tf.group(accuracy_update)


            #Now finally create all the summaries you need to monitor and group them into one summary op.
            tf.summary.scalar('losses/Total_Loss', total_loss)
            tf.summary.scalar('accuracy', accuracy)
            tf.summary.scalar('learning_rate', lr)

            return train_op, accum_ops, zero_ops, global_step, metrics_op, variables_to_restore, predictions, lr, accuracy, total_loss

    def validation_step(input_examples, one_hot_labels):   
        with slim.arg_scope(inception.inception_v3_arg_scope()):
            # here logits is the pre-softmax activations
            logits, end_points = inception.inception_v3(
                input_examples,
                num_classes = NUM_CLASS,
                is_training=False, reuse=True)

            #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
            predictions = tf.argmax(end_points['Predictions'], 1)
            probabilities = end_points['Predictions']
            accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, tf.argmax(one_hot_labels, 1))
            metrics_op = tf.group(accuracy_update)

            #Now finally create all the summaries you need to monitor and group them into one summary op.
            tf.summary.scalar('validation/accuracy', accuracy)

            return metrics_op, accuracy, predictions, probabilities

In [ ]:
with def_graph.as_default() as graph:
    def init_dataset(file_path_pattern, mapping_table, is_training = True):
        dataset = MiniDataSet(file_path_pattern, CATEGORY_NAME_PATH, TOTAL_EXAMPLES, NUM_CLASS, is_training = is_training)
        org_image, org_label = dataset.create_dataset()
        image = preprocess_for_inception(org_image, is_training) # final image to train

        label = one_hot_process(org_label, mapping_table, NUM_CLASS) # final label for training
        # no need for shuffle, DatasetDataProvider do this for us
        batch_images, batch_labels = tf.train.batch([image, label], BATCH_SIZE,\
                                            num_threads = INPUT_THREADS,\
                                            capacity = 2000 + 3 * BATCH_SIZE,\
                                            allow_smaller_final_batch = is_training, name = is_training and 'train_batch' or 'validation_batch')
        
        return batch_images, batch_labels

In [ ]:
with def_graph.as_default() as graph:
    mapping_strings = tf.constant( [ str(key) for key in cvt_csv2tfrecord().keys() ] )
    mapping_table = tf.contrib.lookup.index_table_from_tensor(mapping=mapping_strings, default_value=0)
    batch_images, batch_labels = init_dataset(TRAIN_PATH + "output_file*.tfrecords", mapping_table)
    batch_val_images, batch_val_labels = init_dataset(VAL_PATH + "test_output_file*.tfrecords", mapping_table, False)
    with tf.device('/gpu:0'):
        train_op, accum_op, zero_op, global_step, metrics_op, variables_to_restore, pred_op, lr, accuracy, total_loss = train_step(batch_images, batch_labels)
        val_metrics_op, val_accuracy, val_predictions, val_probabilities = validation_step(batch_val_images, batch_val_labels)
        real_val_label = tf.argmax(batch_val_labels, 1)
        
    global_step_zero = global_step.assign(tf.zeros_like(global_step))
                    
    summary_op = tf.summary.merge_all()
    # Create a saver that restores only the pre-trained variables.
    # we have change optim, restore all param use pretrained mode
    #pre_train_saver = tf.train.Saver(variables_to_restore)
    
    variables = slim.get_variables_to_restore()
    restore_from_pretrained = tf.contrib.framework.filter_variables(
        variables,
        include_patterns=None,
        exclude_patterns=['ExponentialMovingAverage', 'accumulate_factor', 'Momentum', 'InceptionV3/Logits', 'InceptionV3/AuxLogits'])

    pre_train_saver = tf.train.Saver(variables_to_restore)
    # Define an init function that loads the pretrained checkpoint.
    # sess is the managed session passed by Supervisor
    def load_pretrain(sess):
        pre_train_saver.restore(sess, PRETRAINED_MODEL_PATH)

    # no need for specify local_variables_initializer and tables_initializer, Supervisor will do this via default local_init_op
    # init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer())
    init_op = tf.group(tf.global_variables_initializer())
    # Pass the init function to the supervisor.
    # - The init function is called _after_ the variables have been initialized by running the init_op.
    # - use default tf.Saver() for ordinary save and restore
    # - save checkpoint every 1.3 hours(4800)
    # - manage summary in current process by ourselves for memory saving
    # - no need to specify global_step, supervisor will find this automately
    # - initialize order: checkpoint -> local_init_op -> init_op -> init_func
    sv = tf.train.Supervisor(logdir=LOG_PATH, init_fn = load_pretrain, init_op = init_op, summary_op = None, save_model_secs=12000, checkpoint_basename='inception_v3_model.ckpt')
    
    final_loss = 0.
    final_accuracy = 0.
    training_state = True

    config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
    #config.gpu_options.allow_growth = True
    with sv.managed_session(config=config) as sess:
    #with sv.prepare_or_wait_for_session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:

        #sess.run(global_step_zero)
        # Here sess was either initialized from the pre-trained-checkpoint or
        # recovered from a checkpoint saved in a previous run of this code.
        for step in range(int(num_steps_per_epoch * NUM_EPOCHES)):       
            if sv.should_stop():
                tf_logging.info('Supervisor emit finished!')
                tf_logging.info('Current Loss: %s', loss)
                tf_logging.info('Current Accuracy: %s', accuracy)
                tf_logging.info('Saving current model to disk(maybe invalid).')
                training_state = False
                break

            start_time = time.time()

            if step % 1000 == 0:
                summ = sess.run(summary_op)
                sv.summary_computed(sess, summ)
                if step > EPOCHES_OVER * num_steps_per_epoch:
                    raise StopIteration("over epoches reached.")

            if step % VAL_CHECK_FREQ == 0:
                with tf.device('/gpu:0'):
                    _, val_acc, val_pred, val_prob, real_label = sess.run([val_metrics_op, val_accuracy, val_predictions, val_probabilities, real_val_label])
                time_elapsed = time.time() - start_time
                tf_logging.info('Validation Speed: {:5.3f}sec/batch'.format(time_elapsed))
                tf_logging.info('Current Streaming ValAccuracy: {:5.3f}%'.format(val_acc*100.))
                tf_logging.info('Real Label: {}'.format(real_label))
                tf_logging.info('Pred Label: {}'.format(val_pred))

            else:
                with tf.device('/gpu:0'):
                    # accumulate gradient to get bigger batch_size
                    sess.run(zero_op)
                    for _ in range(1, ACCUMULATE_STEP):
                        sess.run([accum_op, total_loss])
                    _, _, _, cur_loss, cur_acc, total_step, cur_lr = sess.run([train_op, accum_op, metrics_op, total_loss, accuracy, global_step, lr])
#                     sess.run([train_op])
                time_elapsed = time.time() - start_time

                if step % 10 == 0:
                    final_loss = cur_loss
                    final_accuracy = cur_acc
                    tf_logging.info('Current Speed: {:5.3f}sec/batch'.format(time_elapsed))
                    tf_logging.info('Current Streaming Accuracy: {:5.3f}%'.format(cur_acc*100.))
                    tf_logging.info('Current Loss: {:5.3f}'.format(cur_loss))
                    tf_logging.info('Epoch %s/%s, Global Step: %s', int(total_step / num_steps_per_epoch + 1), NUM_EPOCHES, total_step)
                    tf_logging.info('Current Learning Rate: {}'.format(cur_lr))
      
        if training_state:
            #We log the final training loss and accuracy
            tf_logging.info('Final Loss: %s', final_loss)
            tf_logging.info('Final Accuracy: %s', final_accuracy)
            # Once all the training has been done, save the log files and checkpoint model
            tf_logging.info('Finished training! Model saved.')
        sv.saver.save(sess, sv.save_path, global_step = sv.global_step)
    

INFO:tensorflow:Restoring parameters from /media/rs/0E06CD1706CD0127/Kapok/kaggle/inception-v3/20160828/inception_v3.ckpt
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path /media/rs/0E06CD1706CD0127/Kapok/kaggle/logs/inception_v3_model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Validation Speed: 15.979sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 0.000%
INFO:tensorflow:Real Label: [3186  236 3778 4045 1453 3776 2821 4160 4023 2884 3623 1974 4835 4350 4159
 4583 2923 4429 4792 4350 4279 4970 2816 1644  991 3501 1308 4279 1866 1282
 1301  289 4279 3186 2499 3512 3800 1394 4393 4873  353 2761  396 3412 2736
 3571 3069  140 2884 3692 4970 3623 4153 5090 4279  457 3379  457 1301 4245
 2836  457 3623 2921 2938 3877 3807 3671  200  289  219 4045 3643 2831 2159
 3920 3730 4429 4722 3279 4393  414 3631  672  991 4816 3501 1056 2647 3186
 4279  289 3330 2159 3465 4814  232 2264 3328  289 1261 3929

INFO:tensorflow:Pred Label: [1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899
 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899
 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899
 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899
 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899
 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899
 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899
 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899 1899
 1899 1899 1899 1899 1899 1899 1899 1899]
INFO:tensorflow:Current Speed: 2.526sec/batch
INFO:tensorflow:Current Streaming Accuracy: 11.098%
INFO:tensorflow:Current Loss: 7.956
INFO:tensorflow:Epoch 1/7, Global Step: 157
INFO:tensorflow:Current Learning Rate: 0.0010000000474974513
INFO:tensorflow:Current Speed: 2.499sec/batch
INFO:tensorflow:Current Streaming Acc

INFO:tensorflow:Current Learning Rate: 0.0010000000474974513
INFO:tensorflow:Current Speed: 2.252sec/batch
INFO:tensorflow:Current Streaming Accuracy: 14.457%
INFO:tensorflow:Current Loss: 6.975
INFO:tensorflow:Epoch 1/7, Global Step: 314
INFO:tensorflow:Current Learning Rate: 0.0010000000474974513
INFO:tensorflow:Current Speed: 2.336sec/batch
INFO:tensorflow:Current Streaming Accuracy: 14.660%
INFO:tensorflow:Current Loss: 7.133
INFO:tensorflow:Epoch 1/7, Global Step: 324
INFO:tensorflow:Current Learning Rate: 0.0010000000474974513
INFO:tensorflow:global_step/sec: 0.425
INFO:tensorflow:Current Speed: 2.690sec/batch
INFO:tensorflow:Current Streaming Accuracy: 14.769%
INFO:tensorflow:Current Loss: 7.001
INFO:tensorflow:Epoch 1/7, Global Step: 334
INFO:tensorflow:Current Learning Rate: 0.0010000000474974513
INFO:tensorflow:Validation Speed: 0.154sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 0.446%
INFO:tensorflow:Real Label: [ 762  439 3345  398 3107  689 3835 4970 4816 4118 4

INFO:tensorflow:Current Streaming ValAccuracy: 0.469%
INFO:tensorflow:Real Label: [4816 4703 4236 3768 3623 4957 2849 4429 1607 4792 1607 4393  453 4810 1005
 2938 4309 2902 1411 2942 3501 3698 4800 3054  349 4815 3892  790 2087  229
  232 2902 4274 2938 5229 3165  662  202 1219 4815 4153 3191 3050 1607  991
 3692 3821  991 5254 4153 2277 4350 3623 3643  289 4564  636 2884  768  133
 4480 3571 3328 2717 3262 3623 4792 4179 4045 3368 2781 3643 2904 4264 3191
 3168 3059 3692 1819 3262 4814 4393  361 1435 1012 2894 1644  457 1607 2475
 4812 4780   71  178  289  289 1407  256 2816  927 2897 4816 3277 3929  895
 2886 3768 4153  443 4045 4970  991 4814 1790 3800 2938  377 1892 2552 3569
 4814 3414 3279 4769  886 1118 4816 4350]
INFO:tensorflow:Pred Label: [1277 1277 4861 1277 1277 2499 1277 1277 1277 1277 4861  790 2499 1277 1277
  140 3892 1277 1277 2499 1277 1277 1277 1277 1420 1435 1277 1277 4861 4861
 1277 3869 2499 1277 1277 1435 4861 2499 4861 3819 1277 1899 3892 1277 1277
 3892 1277 1

INFO:tensorflow:Current Speed: 2.500sec/batch
INFO:tensorflow:Current Streaming Accuracy: 18.277%
INFO:tensorflow:Current Loss: 5.908
INFO:tensorflow:Epoch 1/7, Global Step: 647
INFO:tensorflow:Current Learning Rate: 0.0010000000474974513
INFO:tensorflow:Current Speed: 2.433sec/batch
INFO:tensorflow:Current Streaming Accuracy: 18.359%
INFO:tensorflow:Current Loss: 6.834
INFO:tensorflow:Epoch 1/7, Global Step: 657
INFO:tensorflow:Current Learning Rate: 0.0010000000474974513
INFO:tensorflow:Current Speed: 2.465sec/batch
INFO:tensorflow:Current Streaming Accuracy: 18.384%
INFO:tensorflow:Current Loss: 6.986
INFO:tensorflow:Epoch 1/7, Global Step: 667
INFO:tensorflow:Current Learning Rate: 0.0010000000474974513
INFO:tensorflow:Current Speed: 2.448sec/batch
INFO:tensorflow:Current Streaming Accuracy: 18.511%
INFO:tensorflow:Current Loss: 5.788
INFO:tensorflow:Epoch 1/7, Global Step: 677
INFO:tensorflow:Current Learning Rate: 0.0010000000474974513
INFO:tensorflow:Validation Speed: 0.154sec/b

...

INFO:tensorflow:Current Speed: 2.170sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.180%
INFO:tensorflow:Current Loss: 4.246
INFO:tensorflow:Epoch 2/7, Global Step: 26862
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Current Speed: 2.155sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.183%
INFO:tensorflow:Current Loss: 4.065
INFO:tensorflow:Epoch 2/7, Global Step: 26872
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Current Speed: 2.180sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.185%
INFO:tensorflow:Current Loss: 4.070
INFO:tensorflow:Epoch 2/7, Global Step: 26882
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Current Speed: 2.207sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.187%
INFO:tensorflow:Current Loss: 4.086
INFO:tensorflow:Epoch 2/7, Global Step: 26892
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Validation Speed: 0.

INFO:tensorflow:Validation Speed: 0.138sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 38.505%
INFO:tensorflow:Real Label: [2722 2923 1301  498 4361 1411 4274 5225 4160   53 1245 3929 3339 5130 1644
 2501 3328 1255 4861 3390 4814 3773  289 1435 4800 4350 4970 1245 2853 2884
  504 2884 2647 3785  790 4876  457 2897 4153 1411 1435 4816 1766 1324 4861
  345 4305 1435 2159 2902 4350  322 1601  232  790 4792   96 1916  289 1071
   98  337 1239  377  457   80 3773  125   52 4199 2618 1236 4393 3328 1544
 3277 4153 1008 4861   87 2675 3533  348  873 2173  191  606  241  437 4253
 2884 2908 2849 4747   58 4153  845 2841 1706 2908 3262 2902 1304 2884 2781
 4815 4562 5011 4792 2878 3414  232 1607 1607 3277 1245 4027 2837 2995 4153
  229 3294  377 3277 4960 4279  457 4495]
INFO:tensorflow:Pred Label: [3929 2923 4582  232 2048 1411 4274 3279 4160   53 1245 3929 1268 4792 1644
 4104 3328 4393 4861 3390 4814 3773  289 1435 4800 3329 4429 1245 1377 2884
 4314 3944 2647 3785 3692 4876 3785 28

INFO:tensorflow:Current Speed: 2.142sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.250%
INFO:tensorflow:Current Loss: 3.177
INFO:tensorflow:Epoch 2/7, Global Step: 27205
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Current Speed: 2.150sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.252%
INFO:tensorflow:Current Loss: 4.353
INFO:tensorflow:Epoch 2/7, Global Step: 27215
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Current Speed: 2.184sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.254%
INFO:tensorflow:Current Loss: 4.012
INFO:tensorflow:Epoch 2/7, Global Step: 27225
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Current Speed: 2.191sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.255%
INFO:tensorflow:Current Loss: 4.008
INFO:tensorflow:Epoch 2/7, Global Step: 27235
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Validation Speed: 0.

INFO:tensorflow:Validation Speed: 0.143sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 38.657%
INFO:tensorflow:Real Label: [4814 1637  845 3623  662 3465 2886 3228 4153 1092  341 3748  587 5254 4393
   59 1607 4566 3182 5153 4350 3443 3824 4429  803 1607 4153 2923 4839  389
  284 2923 3415  356 5091 1372 1435 2878 1625  747 4160 3797 3800 2884  322
 1425  689  377  991   98 4350 4429 3279 1435 1539 2715 2849  473 1646 3886
 2821 3280 5210 4429 1864 2232 4689  790 3040 2840 2613 3225 3869 1245 4302
 4582 1684 3159 1411 3330 3940 4160 3785 3141 4429 3115  469  689 4393 1435
 1505 3182  457 2894 3165 3262 3741 2341 3186 1762 3784 4498 1770 3379 4800
 3800 3165 3797 3623  146 5210 1876 1505   47 2821 2923 2178 3414 1148 1219
  984 3593 3279 3571  916 3643 2764 2784]
INFO:tensorflow:Pred Label: [4814 1601  762 3262  689 3465 2886 3280 4160  756 4792 3748 4045 5254 4393
  115 1607 4566 3182 3362 4350 5039 1277 4429 3330 1607 4153 2923 4839  317
 5130 2923 3415 2887 5091 1372 1435 28

INFO:tensorflow:Current Speed: 2.230sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.320%
INFO:tensorflow:Current Loss: 4.267
INFO:tensorflow:Epoch 2/7, Global Step: 27548
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Current Speed: 2.173sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.321%
INFO:tensorflow:Current Loss: 4.033
INFO:tensorflow:Epoch 2/7, Global Step: 27558
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Current Speed: 2.123sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.323%
INFO:tensorflow:Current Loss: 4.327
INFO:tensorflow:Epoch 2/7, Global Step: 27568
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Current Speed: 2.095sec/batch
INFO:tensorflow:Current Streaming Accuracy: 41.326%
INFO:tensorflow:Current Loss: 3.771
INFO:tensorflow:Epoch 2/7, Global Step: 27578
INFO:tensorflow:Current Learning Rate: 1.679616434557829e-05
INFO:tensorflow:Validation Speed: 0.